In [1]:
import basedosdados as bd
import pandas as pd

In [34]:
query = """
SELECT *
FROM `basedosdados.br_inmet_bdmep.microdados` 
LIMIT 100
"""

df = bd.read_sql(query, billing_project_id="python-371123")

Downloading: 100%|██████████| 100/100 [00:00<00:00, 221.07rows/s]


In [36]:
df

,ano,data,hora,id_estacao,precipitacao_total,pressao_atm_hora,pressao_atm_max,pressao_atm_min,radiacao_global,temperatura_bulbo_hora,...,temperatura_max,temperatura_min,temperatura_orvalho_max,temperatura_orvalho_min,umidade_rel_max,umidade_rel_min,umidade_rel_hora,vento_direcao,vento_rajada_max,vento_velocidade
0,2000,2000-07-04,20:00:00,A401,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,2001-08-31,19:00:00,A001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,2001-12-07,03:00:00,A801,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,2002-11-01,18:00:00,A603,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,2002-12-18,23:00:00,A401,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019,2019-06-08,14:00:00,A048,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,2019,2019-08-26,18:00:00,A048,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,2016,2016-06-03,23:00:00,A709,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,2016,2016-08-08,21:00:00,A709,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
query = """
SELECT precipitacao_total, data, hora, id_estacao
FROM `basedosdados.br_inmet_bdmep.microdados`
WHERE EXTRACT(MONTH FROM data) = 10
  AND EXTRACT(YEAR FROM data) IN (2022)
  AND EXTRACT(HOUR FROM hora) >= 7
  AND EXTRACT(HOUR FROM hora) <= 17
LIMIT 100000
"""

df = bd.read_sql(query, billing_project_id="python-371123")

Downloading: 100%|██████████| 100000/100000 [00:10<00:00, 9374.43rows/s]


In [39]:
df

,precipitacao_total,data,hora,id_estacao
0,NaN,2022-10-01,14:00:00,A748
1,NaN,2022-10-01,11:00:00,A216
2,NaN,2022-10-01,07:00:00,A020
3,NaN,2022-10-01,16:00:00,A230
4,NaN,2022-10-01,13:00:00,A869
...,...,...,...,...
99995,0.0,2022-10-31,07:00:00,A320
99996,0.0,2022-10-31,10:00:00,A632
99997,0.0,2022-10-31,11:00:00,A929
99998,0.0,2022-10-31,11:00:00,A375


In [33]:
df.hora.sort_values().unique()

<TimeArray>
[ datetime.time(7, 0),  datetime.time(8, 0),  datetime.time(9, 0),
 datetime.time(10, 0), datetime.time(11, 0), datetime.time(12, 0),
 datetime.time(13, 0), datetime.time(14, 0), datetime.time(15, 0),
 datetime.time(16, 0), datetime.time(17, 0)]
Length: 11, dtype: dbtime